In [ ]:
# 引用专利分析
import pandas as pd
import numpy as np

excel_path = '引用（1-5000）.xls'
d = pd.read_excel(excel_path, sheet_name='sheet1') #文件中只有sheet1有数据，直接导入。

In [ ]:
d.set_index(['公开(公告)号'], inplace=True,drop=False) #设置索引

In [ ]:
d = d['CN1066695C':'CN1104724C'] #定位数据
d.index = range(len(d)) #重置索引
d

In [ ]:
#地址处理

import jieba
#导入标准国家名称列表
countrypd = pd.read_csv('country.csv')
countryli = list(countrypd['value'])

placetext = d['第一申请人地址']
placetext

In [ ]:
# 非中国国别处理

matched_placelist  = []
for place in list(placetext):#地址匹配
    status = 0
    if status == 0 and place[-1].encode('UTF-8').isalpha():
        matched_country = place[-2:]
        matched_placelist.append(matched_country)
        status = 1
    else:
        seg_place = list(jieba.cut(place,cut_all=True))
        
        for seg_word in seg_place:#分词匹配            
            if status == 0 and len(seg_word)>1:#判断是否已经匹配
                for country in countryli:
                    if seg_word in country and status==0:
                        matched_placelist.append(country)
                        status = 1 #匹配成功
    
    if status == 0:
        matched_placelist.append(np.nan)
    
matched_placelist

In [ ]:
countrydf = pd.DataFrame(matched_placelist,columns = ['country'],index=d.index)
countrydf

In [ ]:
#处理英文缩略词
countrypd = pd.read_csv('country.csv') #国家中英文对照
countrypd.set_index(['id'],inplace=True) 
countrydic = countrypd.to_dict()['value']
def matchfun(x):
    if x in countrydic and x != 'nan':
        return countrydic[x]
    else:
        return x
countrydf = countrydf['country'].apply(matchfun)

In [ ]:
d = d.join(countrydf)
d

In [ ]:
d.to_excel('citewithcountry.xls',sheet_name='sheet1')
# 等待手工检验

In [ ]:
###注意：以下为经人工筛选后的数据

#筛选目标的原专利至do （original）
do = d[d['公开(公告)号'].notnull()]
do

In [ ]:
#筛选所有的引用目标专利的专利数据至dc（cite）
d = pd.read_excel('citewithcountry.xls', sheet_name='sheet1')
d['公开(公告)号'] = d['公开(公告)号'].fillna(method='ffill') #填充公开号中的空行
dc = d[d["引用专利"].notnull()]
dc

In [ ]:
# 导出至cite.xls
writer = pd.ExcelWriter('citewithcountry.xls')

d.to_excel(writer,'sheet1')
do.to_excel(writer,'original')
dc.to_excel(writer,'citeby')

writer.save()